In [2]:
import pandas as pd
import numpy as np


In [4]:
!pip install chardet


In [4]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

nltk.download('stopwords')

# Initialize the stemmer
ps = PorterStemmer()

# Define a function for text preprocessing
def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text if word not in stopwords.words('english')]
    return ' '.join(text)

# Apply the function to the message column
df['v2'] = df['v2'].apply(preprocess_text)
df.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,go jurong point crazi avail bugi n great world...,NaN,NaN,NaN
1,ham,ok lar joke wif u oni,NaN,NaN,NaN
2,spam,free entri wkli comp win fa cup final tkt st m...,NaN,NaN,NaN
3,ham,u dun say earli hor u c alreadi say,NaN,NaN,NaN
4,ham,nah think goe usf live around though,NaN,NaN,NaN


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
tfidf = TfidfVectorizer(max_features=5000)

# Fit and transform the message column
X = tfidf.fit_transform(df['v2']).toarray()

# Encode the labels
y = pd.get_dummies(df['v1'], drop_first=True).values.ravel()

print(X.shape, y.shape)


(5572, 5000) (5572,)


In [13]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(4457, 5000) (1115, 5000) (4457,) (1115,)


In [14]:
from sklearn.naive_bayes import MultinomialNB

# Initialize the model
model = MultinomialNB()

# Train the model
model.fit(X_train, y_train)


MultinomialNB()

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.968609865470852
Precision: 1.0
Recall: 0.7666666666666667
F1 Score: 0.8679245283018869


In [16]:
# Function to classify new emails
def classify_email(email):
    email = preprocess_text(email)
    email_tfidf = tfidf.transform([email]).toarray()
    prediction = model.predict(email_tfidf)
    return 'spam' if prediction == 1 else 'ham'

# Test the classifier with a sample email
sample_email = "Congratulations! You've won a $1,000 Walmart gift card. Click here to claim your prize."
print(classify_email(sample_email))


spam
